# 结论 2023-12-07
和 dynamo export 一致

1. 不支持 Tensor 作为控制条件
2. 只支持静态控制场景

In [2]:
import torch

In [12]:
def fn(x):
    if x.sum() > 0:
        return torch.relu(x)
    return torch.sigmoid(x)


flag = torch.randn(1) > 0.5 # failed, Dynamic control flow is not supported at the moment
flag = torch.randn(1).item() > 0.5

def static_flag_fn(x):
    if flag:
        return torch.relu(x)
    return torch.sigmoid(x)

x = torch.randn(3)
out = torch.export.export(static_flag_fn, (x,))
print(out)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, arg0_1: f32[3]):
            # 
            sigmoid: f32[3] = torch.ops.aten.sigmoid.default(arg0_1);  arg0_1 = None
            return (sigmoid,)
            
Graph Signature: ExportGraphSignature(parameters=[], buffers=[], user_inputs=['arg0_1'], user_outputs=['sigmoid'], inputs_to_parameters={}, inputs_to_buffers={}, buffers_to_mutate={}, backward_signature=None, assertion_dep_token=None)
Symbol to range: {}



In [4]:
torch.export.export(fn, 

Signature:
torch.export.export(
    f: Callable,
    args: Tuple[Any, ...],
    kwargs: Union[Dict[str, Any], NoneType] = None,
    *,
    constraints: Union[List[torch.export.Constraint], NoneType] = None,
) -> torch.export.ExportedProgram
Docstring:
:func:`export` takes an arbitrary Python callable (an nn.Module, a function or
a method) and produces a traced graph representing only the Tensor
computation of the function in an Ahead-of-Time (AOT) fashion, which can
subsequently be executed with different outputs or serialized.  The traced
graph (1) produces a normalized operator set consisting only of functional
`Core ATen Operator Set <https://pytorch.org/docs/stable/ir.html>`_
and user specified custom operators, (2) has eliminated all Python control
flow and data structures (except for certain
conditions), and (3) has the set of shape constraints needed to show that
this normalization and control flow elimination is sound for a future
input.

**Soundness Guarantee**

While tracing,